In [1]:
import h2o
from h2o.estimators.gbm import H2OGradientBoostingEstimator

In [2]:
# Connect to a pre-existing cluster
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O cluster uptime:,17 mins 14 secs
H2O cluster timezone:,America/Denver
H2O data parsing timezone:,UTC
H2O cluster version:,3.18.0.2
H2O cluster version age:,"7 days, 20 hours and 29 minutes"
H2O cluster name:,H2O_from_python_jpitt_w07d4a
H2O cluster total nodes:,1
H2O cluster free memory:,3.542 Gb
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster status:,"locked, healthy"


In [3]:
from h2o.utils.shared_utils import _locate # private function. used to find files within h2o git project directory.

df = h2o.import_file(path=_locate("train.csv"))

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [5]:
df.describe()

Rows:10886
Cols:12




,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
type,time,int,int,int,int,real,real,int,real,int,int,int
mins,1293840000000.0,1.0,0.0,0.0,1.0,0.82,0.76,0.0,0.0,0.0,0.0,1.0
mean,1324965382399.412,2.5066139996325556,0.02856880396839978,0.6808745177291935,1.418427337865148,20.230859819952233,23.655084052911995,61.88645967297445,12.799395406944695,36.02195480433584,155.55217710821236,191.57413191254824
maxs,1355958000000.0,4.0,1.0,1.0,4.0,41.0,45.455,100.0,56.9969,367.0,886.0,977.0
sigma,18187689411.512825,1.116174309344325,0.16659885062470955,0.4661591687997357,0.6338385858190958,7.791589843987567,8.474600626484944,19.245033277394686,8.164537326838689,49.960476572649526,151.03903308192454,181.1444538302853
zeros,0,0,10575,3474,0,0,0,22,1313,986,15,0
missing,0,0,0,0,0,0,0,0,0,0,0,0
0,2011-01-01 00:00:00,1.0,0.0,0.0,1.0,9.84,14.395,81.0,0.0,3.0,13.0,16.0
1,2011-01-01 01:00:00,1.0,0.0,0.0,1.0,9.02,13.635,80.0,0.0,8.0,32.0,40.0
2,2011-01-01 02:00:00,1.0,0.0,0.0,1.0,9.02,13.635,80.0,0.0,5.0,27.0,32.0


In [12]:
# Remove things from training frame
train = df.drop(["datetime", "registered", "casual"])

# Convert CAPSULE to a logical factor
train['holiday'] = train['holiday'].asfactor()
train['workingday'] = train['workingday'].asfactor()
train['season'] = train['season'].asfactor()
train['weather'] = train['weather'].asfactor()

In [13]:
# See that the data is ready
train.describe()

Rows:10886
Cols:9




,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,count
type,enum,enum,enum,enum,real,real,int,real,int
mins,,,,,0.82,0.76,0.0,0.0,1.0
mean,,,,,20.230859819952233,23.655084052911995,61.88645967297445,12.799395406944695,191.57413191254824
maxs,,,,,41.0,45.455,100.0,56.9969,977.0
sigma,,,,,7.791589843987567,8.474600626484944,19.245033277394686,8.164537326838689,181.1444538302853
zeros,,,,,0,0,22,1313,0
missing,0,0,0,0,0,0,0,0,0
0,1,0,0,1,9.84,14.395,81.0,0.0,16.0
1,1,0,0,1,9.02,13.635,80.0,0.0,40.0
2,1,0,0,1,9.02,13.635,80.0,0.0,32.0


In [35]:
# Run GBM
my_gbm = H2OGradientBoostingEstimator(distribution = "gamma", ntrees=500, learn_rate=.01)

my_gbm.train(x=list(range(0,train.ncol - 1)), y="count", training_frame=train, validation_frame=train)

gbm Model Build progress: |███████████████████████████████████████████████| 100%


In [36]:
my_gbm_metrics = my_gbm.model_performance(train)
my_gbm_metrics.show()


ModelMetricsRegression: gbm
** Reported on test data. **

MSE: 20248.8914263739
RMSE: 142.29859952358595
MAE: 103.21973945400408
RMSLE: 1.2555516806424891
Mean Residual Deviance: 12.10990516315554
